# Задача 1

Пусть $X_1, X_2, \ldots, X_n$ — выборка из экспоненциального распределения с параметром $\lambda$. Найти оценку максимального правдоподобия параметра $\lambda$, сравнить ее с байесовской оценкой (MAP и математическое ожидание апостреорного распределения), подобрав сопряженное распределение. Сравнить полученные байесовские оценки с оценкой MLE. Найти предсказательное распределение

### Решение задачи 1

Работаем с экспоненциальным распределением и показываем, чем отличаются оценки: частотная MLE, байесовские MAP и математическое ожидание апостериорного распределения. Ниже все формулы и короткая проверка на игрушечном наборе данных.

**Аналитические шаги**

- Плотность экспоненциального распределения: $p(x \mid \lambda) = \lambda e^{-\lambda x},\, x>0$. Правдоподобие выборки $L(\lambda)=\lambda^n e^{-\lambda \sum x_i}$.
- Логарифм: $\ell(\lambda)= n\ln \lambda - \lambda \sum x_i$. Производная и приравнивание к нулю дают оценку максимального правдоподобия $\hat{\lambda}_{	ext{MLE}} = n/\sum x_i$.
- Сопряжённый приор для параметра интенсивности $\lambda$ — гамма-распределение $	ext{Gamma}(lpha, eta)$ (параметризация через `shape` $lpha$ и `rate` $eta$): $p(\lambda)=	frac{eta^{lpha}}{\Gamma(lpha)}\lambda^{lpha-1}e^{-eta\lambda}$.
- Апостериорное распределение после наблюдения выборки остаётся гамма-распределением: $\lambda \mid X \sim 	ext{Gamma}(lpha+n, eta+\sum x_i)$.
- Байесовские оценки: MAP $= 	frac{lpha+n-1}{eta+\sum x_i}$ (при $lpha+n>1$ максимум лежит внутри области), математическое ожидание $E[\lambda\mid X]=	frac{lpha+n}{eta+\sum x_i}$.
- Предсказательное распределение для нового наблюдения $y$ получается интегрированием по $\lambda$: $p(y \mid X)=	frac{lpha_	ext{post}\, eta_	ext{post}^{lpha_	ext{post}}}{(eta_	ext{post}+y)^{lpha_	ext{post}+1}}$, где $lpha_	ext{post}=lpha+n$, $eta_	ext{post}=eta+\sum x_i$. Это распределение Ломакса; его среднее существует при $lpha_	ext{post}>1$ и равно $	frac{eta_	ext{post}}{lpha_	ext{post}-1}$.

In [ ]:
import numpy as np

# Синтетическая выборка: истинная интенсивность 0.6, берём 20 наблюдений
np.random.seed(0)
true_lambda = 0.6
sample = np.random.exponential(scale=1/true_lambda, size=20)

# Параметры гамма-априори (shape, rate)
alpha_prior, beta_prior = 2.0, 1.0

n = len(sample)
sum_x = sample.sum()

# Частотная оценка MLE
lambda_mle = n / sum_x

# Апостериорные параметры
alpha_post = alpha_prior + n
beta_post = beta_prior + sum_x

# Байесовские оценки интенсивности
lambda_map = (alpha_post - 1) / beta_post
lambda_mean = alpha_post / beta_post

# Предсказательная плотность Ломакса на сетке значений
y_grid = np.linspace(0, 8, 5)
pred_density = (alpha_post * (beta_post ** alpha_post)) / ((beta_post + y_grid) ** (alpha_post + 1))

print(f"Истинное значение интенсивности: {true_lambda:.3f}")
print(f"MLE: {lambda_mle:.3f}")
print(f"MAP: {lambda_map:.3f}")
print(f"Мат. ожидание постериора: {lambda_mean:.3f}")
print('Плотность предсказательного распределения на нескольких точках:')
for y, p in zip(y_grid, pred_density):
    print(f'  y={y:.1f} -> p(y|X)={p:.4f}')


**Вывод по задаче 1.** При небольшом количестве наблюдений байесовские оценки тянут интенсивность к априорному среднему, что снижает переобучение. MLE игнорирует приор и может сильно колебаться. Предсказательное распределение автоматически учитывает неопределённость по $\lambda$, распределение хвостатее экспоненциального, поэтому редкие большие значения $y$ получают ненулевую вероятность.

# Задача 2

**Мультиномиальное распределение**

Пусть проводится серия из $n$ испытаний и в результате каждого испытания происходит ровно одно событие из набора $A_1, A_2, \dots, A_m$, причем вероятности этих событий равны соответственно $\mathsf{p}_1, \mathsf{p}_2, \dots, \mathsf{p}_m$, причем
$$\sum_{i=1}^{m}\mathsf{p}_i = 1.$$

Тогда совместное распределение величин $X_1, X_2, \dots, X_m$, где $X_k$ — число наступлений события $A_k$ в серии из $n$ испытаний, задается вероятностями

$$
\mathsf{P}\left(X_1 = n_1, \dots, X_m = n_m, \right) = \frac{n!}{n_1!\dots n_m!}\mathsf{p}_1^{n_1}\dots \mathsf{p}_m^{n_m},
$$

где $n_1, n_2, \dots, n_m$ — произвольный набор целых неотрицательных чисел, таких что

$$\sum_{i=1}^m n_i = n.$$

Произведите байесовский вывод для мультиномиального распределения: найдите апостериорное распределение, используя в качестве сопоряженного распределения к правдоподобию [распределение Дирихле](https://ru.wikipedia.org/wiki/%D0%A0%D0%B0%D1%81%D0%BF%D1%80%D0%B5%D0%B4%D0%B5%D0%BB%D0%B5%D0%BD%D0%B8%D0%B5_%D0%94%D0%B8%D1%80%D0%B8%D1%85%D0%BB%D0%B5), найдите предсказательное распределение. Объясните результат.

### Решение задачи 2

Для мультиномиальной модели сопряжённым приором является распределение Дирихле. Оно удобно тем, что параметры приора складываются с фактическими подсчётами и работают как "псевдонаблюдения".

**Формулы вывода**

- Приор: $\mathbf{p} \sim 	ext{Dir}(lpha_1, \dots, lpha_m)$, где $lpha_i>0$.
- Правдоподобие выборки $\mathbf{n}=(n_1,\dots,n_m)$ с $\sum n_i=n$ пропорционально $\prod p_i^{n_i}$.
- Апостериорное распределение остаётся Дирихле: $\mathbf{p} \mid \mathbf{n} \sim 	ext{Dir}(lpha_1+n_1, \dots, lpha_m+n_m)$.
- Предсказание одного следующего испытания: $\mathsf{P}(A_k \mid \mathbf{n}) = 	frac{lpha_k+n_k}{\sum_i (lpha_i+n_i)}$.
- Предсказание пакета из $t$ испытаний подчиняется распределению Дирихле–мультиномиальному с плотностью "комбинацией из факториалов":
$$p(\mathbf{u} \mid \mathbf{n}) = 
rac{t!}{u_1!\dots u_m!} \
rac{\Gamma\left(\sum_i (lpha_i+n_i)ight)}{\Gamma\left(t+\sum_i (lpha_i+n_i)ight)} 
\prod_{i=1}^m \
rac{\Gamma(lpha_i+n_i+u_i)}{\Gamma(lpha_i+n_i)},$$
где $\mathbf{u}$ — будущие счётчики.
- Интерпретация: большие $lpha_i$ делают распределение более уверенным до наблюдений; нули в данных сглаживаются за счёт псевдосчётчиков.

In [ ]:
import numpy as np
from scipy.special import gammaln

# Четыре исхода, до наблюдений считаем их равновероятными (по 1 псевдосчёту)
alpha_prior = np.array([1., 1., 1., 1.])

# Наблюдённые количества событий
counts = np.array([12, 5, 3, 0])

alpha_post = alpha_prior + counts
prob_next = alpha_post / alpha_post.sum()

print('Апостериорные параметры Дирихле:', alpha_post)
print('Предсказательные вероятности следующего испытания:')
for i, p in enumerate(prob_next, 1):
    print(f'  P(A{i}) = {p:.3f}')

# Пример: вероятность конкретного будущего набора из t=3 испытаний
future_counts = np.array([1, 1, 1, 0])
t = future_counts.sum()
term1 = np.exp(gammaln(t + 1) - gammaln(future_counts + 1).sum())
term2 = np.exp(gammaln(alpha_post.sum()) - gammaln(alpha_post.sum() + t))
term3 = np.exp((gammaln(alpha_post + future_counts) - gammaln(alpha_post)).sum())
p_future = term1 * term2 * term3
print(f'Вероятность набора {future_counts.tolist()} для трёх будущих испытаний: {p_future:.4f}')


**Вывод по задаче 2.** Дирихле даёт удобное обновление: новые наблюдения просто добавляются к параметрам. Предсказательные вероятности для новых испытаний совпадают с относительными частотами, сглаженными псевдосчётчиками. Это убирает нули и делает модель устойчивой при маленьких выборках.